# Preparacion de los datos

En esta seccion realizaremos la lectura del Excel, asi como su limpieza y de la creacion del dataframe con el que trabajaremos

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import seaborn as sns
import numpy as np

Definiremos una funcion para la **lectura** 

In [ ]:
def lectura(path, limit = 250):
    '''
    Parameters
    
    path La ruta del archivo excel
    limit Hasta que universidad tomara
    
    Returns
    
    Un dataframe con todos los años
    
    '''
    df_raw = pd.DataFrame()
    for i in range(2011,2022):
        try:
            df_raw = pd.concat([df_raw, pd.read_excel(path,sheet_name=str(i)).head(limit)])
        except FileNotFoundError:
            print('Archivo no encontrado')
            return None
    return df_raw
        
    

Y una para **limpieza**

In [ ]:
def leer(path,limite = 250):
    '''
    Parameters
    
    path La ruta del archivo excel
    limite Hasta que universidad tomara
    
    Returns
    
    Un dataframe limpio
    
    '''
    df = lectura(path,limite)

    try:
        # Limpiar los rangos
        df['Rank'] = df['Rank'].fillna(method='ffill')
        df['O_Rank'] = pd.to_numeric(df['O_Rank'],errors = 'coerce')
        df['O_Rank'] = df['O_Rank'].fillna(method='ffill')
        df['AR Rank'] = pd.to_numeric(df['AR Rank'],errors = 'coerce')
        df['AR Rank'] = df['AR Rank'].fillna(method='ffill')
        df['ER Rank'] = pd.to_numeric(df['ER Rank'],errors = 'coerce')
        df['ER Rank'] = df['ER Rank'].fillna(method='ffill')
        df['FS Rank'] = df['FS Rank'].fillna(method='ffill')
        df['CF Rank'] = df['CF Rank'].fillna(method='ffill')
        df['IF Rank'] = df['IF Rank'].fillna(method='ffill')
        df['IS Rank'] = df['IS Rank'].fillna(method='ffill')
        # Limpiar las reputaciones
        df['Academic Reputation'] = df['Academic Reputation'].fillna(
            df['Academic Reputation'].mean)
        df['Employer Reputation'] = pd.to_numeric(df['Employer Reputation'],errors = 'coerce')
        df['Employer Reputation'] = df['Employer Reputation'].fillna(
            df['Employer Reputation'].mean())
        # Limpiar faculty
        df['Faculty Student'] = pd.to_numeric(df['Faculty Student'],errors = 'coerce')
        df['Faculty Student'] = df['Faculty Student'].fillna(
            df['Faculty Student'].median())
        df['Citations per Faculty'] = df['Citations per Faculty'].fillna(
            df['Citations per Faculty'].median())
        df['International Students'] = df['International Students'].fillna(
            df['International Students'].median())
        df['International Faculty'] = pd.to_numeric(df['International Faculty'],errors = 'coerce')
        df['International Faculty'] = df['International Faculty'].fillna(
        df['International Faculty'].median())
        # Limpiar el overall
        df['Overall Score'] = df['Overall Score'].fillna(df['Overall Score'].mean())
        df = df.replace('601+', int(601))
    except TypeError:
        print('Error en la lectura')
        return None
    return df
    

Finalmente guardamos el Dataframe

In [ ]:
df = leer('../data/raw/QS WUR 2011-2022.xlsx',400)

In [ ]:
df.shape

In [ ]:
df.isna().sum()

# Modelacion

En esta etapa analizaremos los datos, principalmente 3 analisis

* Regresion lineal multiple, para predecir la importancia de cada Categoria en el *Overall Score*
* Regresion lineal simple, con el Tec para predecir su posicion en los rankings el proximo año
* Importancia de cada **Rank** 

## Regresion lineal multiple

Empezaremos escogiendo las columnas mas importantes, despues separemos los datos en entrenamiento y prueba, para finalmente conseguir el modelo

In [ ]:
df.columns
ranks = ['Rank','O_Rank','AR Rank', 'ER Rank',
       'FS Rank', 'CF Rank', 
       'IF Rank','IS Rank']

Para visualizar la correlacion de cada variable usaremos un **heatmap**

In [ ]:
sns.heatmap(data=df.corr())
plt.show()

In [ ]:
columnas = ['Academic Reputation','Employer Reputation','Faculty Student',
            'Citations per Faculty','International Faculty','International Students']

In [ ]:
X = df[columnas].values

Y = df['Overall Score'].values


Para evitar _overfitting_ separamos los datos en 80% para entrenamiento y 20% para test

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0)


Aplicamos el modelo de regresion lineal con los datos de entrenamiento

In [ ]:
modelo = LinearRegression()


In [ ]:
modelo.fit(X_train, Y_train)


In [ ]:
coeff = pd.DataFrame(modelo.coef_, columnas, columns=['Coeffs'])


In [ ]:
coeff


La variable *Y_pred* sera la y que el modelo predecira usando los datos de test o sea el 20%

In [ ]:
Y_pred = modelo.predict(X_test)


Para la validacion se calculara la diferencia entre la _y_ de prueba y la _y_ predecida

In [ ]:
validacion = pd.DataFrame(
    {"Actual": Y_test, "Prediccion": Y_pred, "Diferencia": Y_test-Y_pred})


Tomamos una muestra de la validacion para hacer nuestros plot

In [ ]:
validacion
muestra = validacion.sample(30)


Adicionalmente calculamos el $R^2$

In [ ]:
r2_train = modelo.score(X_train, Y_train)
r2_test = modelo.score(X_test, Y_test)
print(f"R2 del entrenamiento: {r2_train:.4f}\nR2 del test: {r2_test:.4f}")


Finalmente hacemos este plot, el cual nos muestra una buena precision del modelo

In [ ]:
muestra.plot.bar()
plt.show()

In [ ]:
coeffs = abs(coeff.sort_values(by='Coeffs',ascending=False))
coeffs.plot.bar(figsize=(10,10),color='green')
plt.grid()
plt.show()

### Puntos destacables

1. Las citas por escuela son mas importantesque la reputacion del empleador, el Tec suele ser reconocida por empleadores pero no es un rubro tan importante al parecer
2. La reputacion academica es casi la mitad del score
3. La parte internacional no parece ser tan importante

## Prediccion 2022 del tec

In [ ]:
df_tec = df[df['Institution '] == 'Tecnológico de Monterrey (ITESM)']

In [ ]:
df_tec.shape

In [ ]:
nuevo = [2022,np.nan,np.nan,'Tecnológico de Monterrey (ITESM)',np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,0]
nuevo = pd.Series(nuevo,index=df_tec.columns)

In [ ]:
df_tec = df_tec.append(nuevo, ignore_index=True)

In [ ]:
df_tec = df_tec.interpolate()

In [ ]:
df_tec

In [ ]:
df_tec[columnas].loc[5]

In [ ]:
modelo.predict([df_tec[columnas].loc[5]])

Siguiendo el modelo, al tec le esperaria un score de **48.7**

In [ ]:
df_2021 = df[df['Year']==2021]

In [ ]:
df_2021[df_2021['Overall Score']<50]

Si las condiciones siguen iguales, puede que el tec suba a la posicion **157**

## Importancia de cada rank

Nuevamente una regresion multiple para calcular la importancia de cada rank en el rank final

In [ ]:
ranks = ['AR Rank', 'ER Rank',
       'FS Rank', 'CF Rank', 
       'IF Rank','IS Rank']

In [ ]:
X =  df[ranks].values

In [ ]:
Y = df['Overall Score'].values

Para evitar _overfitting_ separamos los datos en 80% para entrenamiento y 20% para test

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0)


Aplicamos el modelo de regresion lineal con los datos de entrenamiento

In [ ]:
modelo = LinearRegression()


In [ ]:
modelo.fit(X_train, Y_train)


In [ ]:
coeff = pd.DataFrame(modelo.coef_, ranks, columns=['Coeffs'])


In [ ]:
coeff


In [ ]:
r2_train = modelo.score(X_train, Y_train)
r2_test = modelo.score(X_test, Y_test)
print(f"R2 del entrenamiento: {r2_train:.4f}\nR2 del test: {r2_test:.4f}")


In [ ]:
coeffs = abs(coeff.sort_values(by='Coeffs',ascending=True))
coeffs.plot.bar(figsize=(10,10),color='red')
plt.grid()
plt.show()

### Puntos destacables

1. Usando esta metrica, el rubro Faculty Student es mas importante que el Citations per Faculty